# Installation and imports

In [ ]:
!pip install fastbook

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

# Create folders and download data

In [ ]:
path = Path('fruits')
if not path.exists():
  path.mkdir()

In [ ]:
fruit_types = "dragon fruit", "jackfruit", "mangosteen", "durian", "tamarind", "rambutan", "custard apple", "rose apple", "pomelo", "sapodilla", "salak fruit", "thai bananas", "nam dok mai mango"

In [ ]:
for f in fruit_types:
  dest = (path/f)
  dest.mkdir(exist_ok=True)
  results = search_images_ddg(f"{f}", max_images=100)
  download_images(dest, urls=results)

fns = get_image_files(path)
print(fns)

In [ ]:
failed = verify_images(fns)
failed.map(Path.unlink)

# Create Dataloaders

In [ ]:
fruits = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
fruits = fruits.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())

In [ ]:
dls = fruits.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

# Create Learner and fine-tune

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(10, 10))

# Optional step: Data cleaning

In [ ]:
interp.plot_top_losses(5, nrows=5)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()

# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

# Save fine-tuned model

In [ ]:
learn.export()